# LlamaIndex LLM integrations

LlamaIndex exposes a common interface for all their LLM integrations.

---

## **1. What are LLM Integrations in LlamaIndex?**
LlamaIndex acts as a bridge between your data and Large Language Models (LLMs). It allows you to:
- **Index** unstructured/structured data.
- **Query** the indexed data using LLMs.
- **Augment** LLMs with external knowledge.

**LLM Integrations** refer to how LlamaIndex interacts with different LLMs (OpenAI, Hugging Face, Anthropic, etc.) to generate responses, perform retrieval-augmented generation (RAG), or fine-tune models.

---

## **2. Types of LLM Integrations**
We’ll cover:
- OpenAI
- Hugging Face (Local Models)
- Anthropic
- Custom LLMs

---

### **2.1 OpenAI Integration**
**Theory**:
- Use OpenAI’s GPT models (e.g., `gpt-3.5-turbo`, `gpt-4`) via API.
- Handles **text completion**, **chat**, and **embeddings**.

#### **Code Example**:
```python
from llama_index.llms import OpenAI
from llama_index import VectorStoreIndex, SimpleDirectoryReader

# Set API Key
import os
os.environ["OPENAI_API_KEY"] = "your-api-key"

# Initialize LLM
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.7)

# Load data and create index
documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents)

# Query using the LLM
query_engine = index.as_query_engine(llm=llm)
response = query_engine.query("What is the capital of France?")
print(response)
```

**Explanation**:
- `temperature=0.7`: Controls randomness (0 = deterministic, 1 = creative).
- The LLM generates responses based on the indexed data.

---

### **2.2 Hugging Face Integration (Local Models)**
**Theory**:
- Run open-source models locally (e.g., `Llama-2`, `BERT`).
- Useful for privacy, cost savings, or custom fine-tuning.

#### **Code Example**:
```python
from llama_index.llms import HuggingFaceLLM
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load a Hugging Face model
model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Initialize LLM
llm = HuggingFaceLLM(
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    device_map="auto",  # Use GPU if available
)

# Use in a pipeline
from llama_index import ServiceContext
service_context = ServiceContext.from_defaults(llm=llm)

# Build an index and query
index = VectorStoreIndex.from_documents(documents, service_context=service_context)
```

**Explanation**:
- `device_map="auto"`: Automatically uses GPU (CUDA) if available.
- Requires `transformers` and `torch` libraries.

---

### **2.3 Anthropic (Claude) Integration**
**Theory**:
- Use Anthropic’s Claude models for safer, structured outputs.
- Requires API access.

#### **Code Example**:
```python
from llama_index.llms import Anthropic

# Set API Key
os.environ["ANTHROPIC_API_KEY"] = "your-api-key"

# Initialize LLM
llm = Anthropic(model="claude-2", max_tokens=1000)

# Query example
response = llm.complete("Explain quantum mechanics in simple terms.")
print(response.text)
```

**Explanation**:
- `max_tokens`: Limits response length.
- Claude is optimized for helpfulness and harmlessness.

---

### **2.4 Custom LLM Integration**
**Theory**:
- Wrap any LLM (e.g., a proprietary model) into LlamaIndex.
- Subclass `CustomLLM` and implement required methods.

#### **Code Example**:
```python
from llama_index.llms import CustomLLM, CompletionResponse

class MyCustomLLM(CustomLLM):
    def __init__(self):
        self.model = my_custom_model  # Replace with your model

    def complete(self, prompt: str) -> CompletionResponse:
        response = self.model.generate(prompt)
        return CompletionResponse(text=response)

# Initialize and use
custom_llm = MyCustomLLM()
service_context = ServiceContext.from_defaults(llm=custom_llm)
```

**Explanation**:
- Replace `my_custom_model` with your model’s inference logic.

---

## **3. Key Concepts**
### **3.1 ServiceContext**
- Central configuration for LLMs, embeddings, and more.
```python
from llama_index import ServiceContext
service_context = ServiceContext.from_defaults(llm=llm, embed_model="local")
```

### **3.2 Temperature & Token Limits**
- **Temperature**: Higher values (e.g., 0.8) increase creativity.
- **Max Tokens**: Controls response length.

---

## **4. Choosing the Right LLM**
| **LLM**          | **Use Case**                          | **Pros**                          | **Cons**                |
|-------------------|---------------------------------------|-----------------------------------|-------------------------|
| OpenAI GPT        | General-purpose, high-quality outputs | Easy integration, powerful        | Cost, API dependency    |
| Hugging Face      | Privacy, customization                | Free, offline use                 | Resource-intensive      |
| Anthropic Claude  | Safety-critical applications          | Structured outputs                | Limited access          |
| Custom LLM        | Proprietary models                    | Full control                      | Development effort      |

---

## **5. Advanced Integrations**
### **5.1 Multi-Modal LLMs**
- Combine text and images (e.g., GPT-4V).
```python
from llama_index.multi_modal_llms import OpenAIMultiModal
mm_llm = OpenAIMultiModal(model="gpt-4-vision-preview")
response = mm_llm.complete(prompt="Describe this image:", image="image.png")
```

### **5.2 Async & Streaming**
- Non-blocking calls and real-time responses.

```python
# Async query
response = await query_engine.aquery("What is AI?")
# Streaming
for chunk in response.response_gen:
    print(chunk, end="")
```

---

## **6. Resources**
- **LlamaIndex Docs**: https://docs.llamaindex.ai
- **Hugging Face Models**: https://huggingface.co/models
- **OpenAI API Docs**: https://platform.openai.com/docs

---





In [8]:
%pip install -Uq 'llama-index'
%pip install -Uq llama-index-llms-openai
%pip install -Uq llama-index-llms-groq

In [9]:
%pip install -Uq python-dotenv

import os
from dotenv import load_dotenv

if os.path.exists('../.env'):
    load_dotenv('../.env')


In [10]:
!pip freeze | grep "llama-index=="

llama-index==0.12.15


In [11]:
# we will use Groq models to make this free to use. but we still need an API key
import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

In [12]:
# we will be using pprint for logging
from pprint import pprint

## One common interface for all LLMs

You can import several LLMs on LlamaIndex. LlamaIndex exposes a common interface to all LLM integrations.

In [13]:
from llama_index.llms.groq import Groq

llm = Groq(model="gemma2-9b-it")

### The `complete()` method

The `complete()` method is the main method of the interface. It is a text-to-text method.

In [14]:
response = llm.complete("Tell me a joke")
print(response)

Why don't scientists trust atoms? 

Because they make up everything! 😄  



In [15]:
type(response)

llama_index.core.base.llms.types.CompletionResponse

In [16]:
pprint(response.__dict__)

{'additional_kwargs': {},
 'delta': None,
 'logprobs': None,
 'raw': ChatCompletion(id='chatcmpl-a5aadd46-5b40-4269-93a5-804fdc81a47e', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Why don't scientists trust atoms? \n\nBecause they make up everything! 😄  \n", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1738699444, model='gemma2-9b-it', object='chat.completion', service_tier=None, system_fingerprint='fp_10c08bf97d', usage=CompletionUsage(completion_tokens=21, prompt_tokens=13, total_tokens=34, completion_tokens_details=None, prompt_tokens_details=None, queue_time=0.051259129, prompt_time=0.001166607, completion_time=0.038181818, total_time=0.039348425), x_groq={'id': 'req_01jk99041sfth8xnsg8p1xypwk'}),
 'text': "Why don't scientists trust atoms? \n"
         '\n'
         'Because they make up everything! 😄  \n'}


### The `chat()` method

The `chat()` method is a message-based method. It takes in a list of messages as input and returns a message as output. This is the most common interface for chatbots.

In [30]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(role="system", content="You are Sherlock Holmes"),
    ChatMessage(role="user", content="Who framed roger rabbit?"),
]
response = llm.chat(messages)
print(response)

assistant: In the film "Who Framed Roger Rabbit," the character who framed Roger Rabbit is Judge Doom. He is the main antagonist of the movie and orchestrates the plot to frame Roger for the murder of Marvin Acme. Judge Doom's ultimate plan is to destroy Toontown to make way for a freeway, and framing Roger is part of his scheme to achieve this goal.


In [18]:
type(response)

llama_index.core.base.llms.types.ChatResponse

In [19]:
pprint(response.__dict__)

{'additional_kwargs': {'completion_tokens': 36,
                       'prompt_tokens': 46,
                       'total_tokens': 82},
 'delta': None,
 'logprobs': None,
 'message': ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text="Why don't scientists trust atoms? \n\nBecause they make up everything! \n\nElementary, my dear Watson.  *adjusts deerstalker*  \n\n")]),
 'raw': ChatCompletion(id='chatcmpl-f7cb7e80-aa4b-4f10-b6a1-b1801f3d7f43', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Why don't scientists trust atoms? \n\nBecause they make up everything! \n\nElementary, my dear Watson.  *adjusts deerstalker*  \n\n", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1738699444, model='gemma2-9b-it', object='chat.completion', service_tier=None, system_fingerprint='fp_10c08bf97d', usage=CompletionUsage(completion_tokens=36,

### Both methods have a streaming version

In [34]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4o")

response = llm.stream_complete("Tell me a joke")

In [35]:
for token in response:
  print(token.text, end="", flush=True)

WhyWhy don'tWhy don't skeletonWhy don't skeletonsWhy don't skeletons fightWhy don't skeletons fight eachWhy don't skeletons fight each otherWhy don't skeletons fight each other?

Why don't skeletons fight each other?

TheyWhy don't skeletons fight each other?

They don'tWhy don't skeletons fight each other?

They don't haveWhy don't skeletons fight each other?

They don't have theWhy don't skeletons fight each other?

They don't have the gutsWhy don't skeletons fight each other?

They don't have the guts!Why don't skeletons fight each other?

They don't have the guts!

In [22]:
type(response)

generator

In [23]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(role="system", content="You are Sherlock Holmes"),
    ChatMessage(role="user", content="Who framed roger rabbit?"),
]

response = llm.stream_chat(messages)

In [24]:
type(response)

generator

### Structured output

In [25]:
from typing import List
from pydantic import BaseModel, Field


class Song(BaseModel):
    """Data model for a song."""

    title: str
    length_seconds: int


class Album(BaseModel):
    """Data model for an album."""

    name: str
    artist: str
    songs: List[Song]

In [26]:
from llama_index.core.llms import ChatMessage

sllm = llm.as_structured_llm(output_cls=Album)
input_msg = ChatMessage.from_str("Generate an example album from The Shining")

response = sllm.chat([input_msg])

In [27]:
response

ChatResponse(message=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='{"name":"The Shining Soundtrack","artist":"Various Artists","songs":[{"title":"Main Title","length_seconds":180},{"title":"Rocky Mountains","length_seconds":210},{"title":"Lonesome Ghosts","length_seconds":240},{"title":"Midnight, the Stars and You","length_seconds":180},{"title":"The Overlook Waltz","length_seconds":200}]}')]), raw=Album(name='The Shining Soundtrack', artist='Various Artists', songs=[Song(title='Main Title', length_seconds=180), Song(title='Rocky Mountains', length_seconds=210), Song(title='Lonesome Ghosts', length_seconds=240), Song(title='Midnight, the Stars and You', length_seconds=180), Song(title='The Overlook Waltz', length_seconds=200)]), delta=None, logprobs=None, additional_kwargs={})

In [28]:
pprint(response.raw.__dict__)

{'artist': 'Various Artists',
 'name': 'The Shining Soundtrack',
 'songs': [Song(title='Main Title', length_seconds=180),
           Song(title='Rocky Mountains', length_seconds=210),
           Song(title='Lonesome Ghosts', length_seconds=240),
           Song(title='Midnight, the Stars and You', length_seconds=180),
           Song(title='The Overlook Waltz', length_seconds=200)]}


---

## **1. The `complete()` Method**
### **What It Does**:
- **`complete()`** is like asking a **single question** to the LLM and getting a **single answer**.
- It takes a **text prompt** as input and returns a **text completion**.
- Use this when you want a straightforward response without conversation history.

### **Parameters**:
- `prompt`: The text you want the LLM to complete.
- `temperature`: Controls randomness (0 = factual, 1 = creative).
- `max_tokens`: Limits the length of the response.

---

### **Example 1: Basic `complete()` with OpenAI**
```python
from llama_index.llms import OpenAI
import os

os.environ["OPENAI_API_KEY"] = "your-api-key"

# Initialize the LLM
llm = OpenAI(model="gpt-3.5-turbo")

# Use complete()
response = llm.complete("Explain gravity in simple terms.")
print(response.text)
```

**Output**:
```
Gravity is the force that pulls objects toward each other. It's why things fall to the ground and why planets orbit the sun. The more mass an object has, the stronger its gravity.
```

---

### **Example 2: `complete()` with Hugging Face (Local LLM)**
```python
from llama_index.llms import HuggingFaceLLM
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load a local model (e.g., Mistral-7B)
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

# Initialize the LLM
llm = HuggingFaceLLM(model=model, tokenizer=tokenizer)

# Use complete()
response = llm.complete("What is the capital of France?")
print(response.text)
```

**Output**:
```
The capital of France is Paris.
```

---

## **2. The `chat()` Method**
### **What It Does**:
- **`chat()`** is like having a **conversation** with the LLM.
- It uses a **list of messages** (with roles like `user` and `assistant`) to maintain context.
- Ideal for multi-turn dialogues.

### **Parameters**:
- `messages`: A list of `ChatMessage` objects (e.g., `user` asks, `assistant` replies).
- `temperature`, `max_tokens`: Same as `complete()`.

---

### **Example 1: Basic `chat()` with OpenAI**
```python
from llama_index.llms import OpenAI, ChatMessage

llm = OpenAI(model="gpt-3.5-turbo")

# Define the conversation history
messages = [
    ChatMessage(role="user", content="Hi! What’s the weather like today?"),
    ChatMessage(role="assistant", content="I don’t have real-time data. Where are you located?"),
    ChatMessage(role="user", content="I’m in Paris.")
]

# Continue the chat
response = llm.chat(messages=messages)
print(response.message.content)
```

**Output**:
```
In Paris, the weather can vary, but you can check a weather website or app for real-time updates!
```

---

### **Example 2: `chat()` with Anthropic (Claude)**
```python
from llama_index.llms import Anthropic, ChatMessage

llm = Anthropic(model="claude-2")

messages = [
    ChatMessage(role="user", content="Write a poem about the ocean."),
]

response = llm.chat(messages=messages)
print(response.message.content)
```

**Output**:
```
The ocean whispers secrets deep and old,
Its waves a dance of blue and gold...
```

---

## **3. Other Key Methods**

### **3.1 `stream()`: Real-Time Responses**
Generates responses **chunk by chunk** (useful for real-time UIs).

```python
from llama_index.llms import OpenAI

llm = OpenAI()

# Stream the response
response = llm.stream("Tell me a story about a dragon.")
for chunk in response:
    print(chunk.delta, end="")  # Print each chunk as it arrives
```

---

### **3.2 `aretrieve()` and `asynthesize()`: Async Methods**
For non-blocking operations (useful in web apps).

```python
import asyncio
from llama_index.llms import OpenAI

llm = OpenAI()

async def async_query():
    response = await llm.acomplete("Explain async programming.")
    print(response.text)

asyncio.run(async_query())
```

---

### **3.3 `retrieve()`: Fetch Relevant Data**
Retrieves data chunks from your index (used in RAG pipelines).

```python
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)
retriever = index.as_retriever()

# Retrieve relevant data for a query
nodes = retriever.retrieve("What is AI?")
for node in nodes:
    print(node.text)
```

---

### **3.4 `synthesize()`: Combine Retrieved Data + LLM**
Generates a final answer using retrieved data and the LLM.

```python
query_engine = index.as_query_engine()
response = query_engine.synthesize(
    query="What is machine learning?",
    nodes=nodes  # Retrieved data from `retrieve()`
)
print(response.response)
```

---

## **4. Key Parameters Explained**

| **Parameter**   | **Description**                                                                 |
|------------------|---------------------------------------------------------------------------------|
| `temperature`    | Controls randomness. Use `0` for facts (e.g., Q&A), `0.8` for creative writing. |
| `max_tokens`     | Maximum length of the response (e.g., `100` for short answers).                 |
| `top_p`          | Controls diversity (e.g., `0.9` focuses on top 90% probable tokens).            |
| `frequency_penalty` | Reduces repetition of phrases (e.g., `0.5` discourages repetition).           |

---

## **5. When to Use Which Method?**

| **Method**       | **Use Case**                                                                 |
|------------------|-----------------------------------------------------------------------------|
| `complete()`     | Single-turn Q&A, summarization, code generation.                            |
| `chat()`         | Multi-turn conversations (e.g., chatbots, interactive dialogues).           |
| `stream()`       | Real-time applications (e.g., ChatGPT-style typing animations).             |
| `retrieve()`     | Fetching data from your index (without generating a response).              |
| `synthesize()`   | Combining retrieved data with LLM to generate a final answer.               |
| Async Methods    | Building web APIs or apps where non-blocking operations are critical.       |

---

## **6. Troubleshooting Common Issues**
1. **`max_tokens` Too Low**: Increase it if responses are cut off.
   ```python
   response = llm.complete("Explain AI...", max_tokens=500)
   ```
2. **Repetitive Outputs**: Adjust `temperature` or `frequency_penalty`.
   ```python
   response = llm.complete("Write a story...", temperature=0.8, frequency_penalty=0.5)
   ```
3. **API Errors**: Check your API key and internet connection.

---

## **7. Full Workflow Example**
Let’s build a **RAG pipeline** using all methods:

```python
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms import OpenAI

# Step 1: Load data
documents = SimpleDirectoryReader("data").load_data()

# Step 2: Index data
index = VectorStoreIndex.from_documents(documents)

# Step 3: Retrieve relevant data
retriever = index.as_retriever()
nodes = retriever.retrieve("What is quantum computing?")

# Step 4: Synthesize a response
query_engine = index.as_query_engine()
response = query_engine.synthesize(query="What is quantum computing?", nodes=nodes)
print(response.response)
```

---


---


---

## **1. Core LLM Methods**
These methods interact directly with the LLM (e.g., GPT-4, Claude).

### **(a) `complete()`**
- **Purpose**: Generate a **single response** from a **text prompt**.
- **Analogy**: Asking a chef to cook a dish (one-time request).
- **Example**:
  ```python
  from llama_index.llms import OpenAI
  
  llm = OpenAI(model="gpt-3.5-turbo")
  response = llm.complete("Explain the solar system.")
  print(response.text)
  ```
  **Parameters**:
  - `prompt`: Input text.
  - `temperature`: Creativity (0 = strict, 1 = random).
  - `max_tokens`: Max response length.

---

### **(b) `chat()`**
- **Purpose**: Have a **multi-turn conversation** using message history.
- **Analogy**: Chatting with a friend who remembers previous messages.
- **Example**:
  ```python
  from llama_index.llms import ChatMessage
  
  messages = [
      ChatMessage(role="user", content="What’s the capital of France?"),
      ChatMessage(role="assistant", content="Paris. What else can I help with?"),
      ChatMessage(role="user", content="What’s its population?")
  ]
  response = llm.chat(messages)
  print(response.message.content)  # Output: "Around 2.2 million."
  ```

---

### **(c) `stream()`**
- **Purpose**: Stream responses **in real-time** (token by token).
- **Analogy**: Watching a movie frame-by-frame instead of all at once.
- **Example**:
  ```python
  response = llm.stream("Tell me a joke.")
  for chunk in response:
      print(chunk.delta, end="")  # Prints tokens as they arrive
  ```

---

### **(d) Async Methods (`acomplete()`, `achat()`)**
- **Purpose**: Non-blocking versions of `complete()` and `chat()`.
- **Analogy**: Sending an email while doing other tasks.
- **Example**:
  ```python
  import asyncio
  
  async def async_query():
      response = await llm.acomplete("What is async programming?")
      print(response.text)
  
  asyncio.run(async_query())
  ```

---

## **2. Query Engine Methods**
These methods query **indexed data** using the LLM.

### **(a) `query()`**
- **Purpose**: Ask questions about your indexed data.
- **Analogy**: Asking a librarian to find a book and summarize it.
- **Example**:
  ```python
  from llama_index import VectorStoreIndex, SimpleDirectoryReader
  
  # Load data and create index
  documents = SimpleDirectoryReader("data").load_data()
  index = VectorStoreIndex.from_documents(documents)
  
  # Query
  query_engine = index.as_query_engine()
  response = query_engine.query("Summarize the document.")
  print(response.response)
  ```

---

### **(b) `aquery()`**
- **Purpose**: Async version of `query()`.
- **Example**:
  ```python
  async def async_query():
      response = await query_engine.aquery("What’s the main theme?")
      print(response.response)
  
  asyncio.run(async_query())
  ```

---

## **3. Retrieval Methods**
Fetch data from your index without generating a response.

### **(a) `retrieve()`**
- **Purpose**: Fetch relevant data chunks for a query.
- **Analogy**: Using a search engine to get links (not summaries).
- **Example**:
  ```python
  retriever = index.as_retriever(similarity_top_k=3)
  nodes = retriever.retrieve("What is AI?")
  for node in nodes:
      print(node.text)  # Raw text chunks from your data
  ```

---

### **(b) `aretrieve()`**
- **Purpose**: Async version of `retrieve()`.
- **Example**:
  ```python
  async def async_retrieve():
      nodes = await retriever.aretrieve("What is ML?")
      print(nodes)
  ```

---

## **4. Index Construction Methods**
Build and manage indexes (structured representations of your data).

### **(a) `from_documents()`**
- **Purpose**: Create an index from documents (PDFs, text files).
- **Example**:
  ```python
  index = VectorStoreIndex.from_documents(documents)
  ```

---

### **(b) `load_index_from_storage()`**
- **Purpose**: Load a pre-saved index (no need to re-index data).
- **Example**:
  ```python
  from llama_index import StorageContext
  
  storage_context = StorageContext.from_defaults(persist_dir="storage")
  index = load_index_from_storage(storage_context)
  ```

---

## **5. Response Synthesis Methods**
Combine retrieved data with LLM to generate answers.

### **(a) `synthesize()`**
- **Purpose**: Generate a response from retrieved nodes.
- **Analogy**: Writing an essay using highlighted book passages.
- **Example**:
  ```python
  response = query_engine.synthesize(
      query="What is Python?",
      nodes=nodes  # Retrieved data
  )
  print(response.response)
  ```

---

## **6. Advanced Methods**
### **(a) Multi-Modal Methods**
- **Purpose**: Combine text and images.
- **Example**:
  ```python
  from llama_index.multi_modal_llms import OpenAIMultiModal
  
  mm_llm = OpenAIMultiModal(model="gpt-4-vision-preview")
  response = mm_llm.complete("Describe this image:", image="image.jpg")
  ```

---

### **(b) Agents**
- **Purpose**: LLMs that use tools (calculators, APIs).
- **Example**:
  ```python
  from llama_index.agent import OpenAIAgent
  
  agent = OpenAIAgent.from_tools(tools=[calculator_tool])
  response = agent.chat("Calculate 2+2.")
  ```

---

## **7. Summary Table: When to Use Which Method**

| **Method**       | **Use Case**                                                                 |
|------------------|-----------------------------------------------------------------------------|
| `complete()`     | Single-turn Q&A, code generation.                                           |
| `chat()`         | Multi-turn conversations (e.g., chatbots).                                  |
| `stream()`       | Real-time streaming (e.g., typing animations).                              |
| `query()`        | Querying indexed data (e.g., document QA).                                  |
| `retrieve()`     | Fetching raw data chunks (e.g., search without summaries).                  |
| `synthesize()`   | Generating answers from retrieved data (e.g., RAG).                         |
| Async Methods    | Building web apps/APIs.                                                     |
| Agents           | Complex tasks requiring tools (e.g., math, web searches).                   |

---

## **8. Parameters Deep Dive**
### Common Parameters Across Methods:
1. **`temperature`**:
   - `0.0`: Factual responses (e.g., Q&A).
   - `0.7`: Balanced creativity (e.g., stories).
   - `1.0`: Maximum randomness.

2. **`max_tokens`**:
   - Limits response length (e.g., `max_tokens=100` for short answers).

3. **`top_p`**:
   - Controls diversity (e.g., `top_p=0.9` for focused responses).

---

## **9. Full Workflow Example**
Let’s build a **RAG pipeline** using all methods:
```python
# Step 1: Load data
documents = SimpleDirectoryReader("data").load_data()

# Step 2: Build index
index = VectorStoreIndex.from_documents(documents)

# Step 3: Retrieve nodes
retriever = index.as_retriever()
nodes = retriever.retrieve("What is climate change?")

# Step 4: Synthesize a response
query_engine = index.as_query_engine()
response = query_engine.synthesize(query="What is climate change?", nodes=nodes)
print(response.response)
```

---